In [1]:
# --- Imports and constants ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import combinations

# --- User settings ---
MIC_XLSX = Path("Mic_locations.xlsx")
PRESSURE_XLSX = Path("Pressure_signal_received_by_microphones.xlsx")

c = 343.0                  # speed of sound [m/s]
f0 = 2000.0                # frequency of interest [Hz]
focus_z = 1.0              # imaging plane [m]
grid_extent = 0.20         # +/- extent in x and y [m]
grid_step = 0.01           # grid spacing [m]

# Synthetic test parameters
fs_synth = 48000.0         # sampling rate [Hz]
T_synth = 0.08             # duration [s]
src_amp = 0.1              # source amplitude


In [ ]:
# --- Helper functions ---

def load_mic_positions(path: Path) -> np.ndarray:
    """Load microphone coordinates from Excel (in meters)."""
    try:
        xl = pd.ExcelFile(path, engine="openpyxl")
    except ImportError:
        raise ImportError(
            "Missing dependency: openpyxl. Please install it with 'pip install openpyxl'."
        )
    df = xl.parse(xl.sheet_names[0])
    df.columns = [str(c).strip().lower() for c in df.columns]
    xcol = next((c for c in df.columns if c.startswith('x')), None)
    ycol = next((c for c in df.columns if c.startswith('y')), None)
    zcol = next((c for c in df.columns if c.startswith('z')), None)
    if xcol is None or ycol is None:
        raise ValueError("Could not find X/Y columns in mic file.")
    if zcol is None:
        df['z'] = 0.0
        zcol = 'z'
    df = df[[xcol, ycol, zcol]].rename(columns={xcol:'x', ycol:'y', zcol:'z'}).astype(float)
    if df[['x','y','z']].abs().max().max() > 10:  # likely mm → m
        df[['x','y','z']] *= 1e-3
    return df[['x','y','z']].to_numpy()


def load_pressures(path: Path):
    """Load time vector and pressure signals from Excel."""
    xl = pd.ExcelFile(path)
    df = xl.parse(xl.sheet_names[0])
    df.columns = [str(c).strip().lower() for c in df.columns]
    tcol = next((c for c in df.columns if c in ('t','time','time (s)','time[s]','timestamp')), None)
    if tcol is None:
        t = None
        P = df.to_numpy(dtype=float)
    else:
        t = df[tcol].to_numpy(dtype=float)
        P = df.drop(columns=[tcol]).to_numpy(dtype=float)
    return t, P

def estimate_fs(t):
    """Estimate sampling frequency from time vector."""
    if t is None or t.size < 2: return None
    dt = np.median(np.diff(t))
    return 1.0/dt if dt > 0 else None

def hann(N):
    n = np.arange(N)
    return 0.5 - 0.5*np.cos(2*np.pi*n/(N-1))

def complex_projection_at_freq(x, fs, f):
    """Project signal(s) onto complex exponential at frequency f."""
    x = np.asarray(x)
    N = x.shape[0]
    w = hann(N)
    n = np.arange(N)
    expo = np.exp(-1j*2*np.pi*f*n/fs)
    if x.ndim == 1:
        num = np.sum((x * w) * expo)
    else:
        num = np.sum((x * w[:,None]) * expo[:,None], axis=0)
    return num / np.sum(w)


In [3]:
# --- Grid and steering functions ---

def make_grid(extent, step, z0):
    x = np.arange(-extent, extent + step/2, step)
    y = np.arange(-extent, extent + step/2, step)
    X, Y = np.meshgrid(x, y, indexing='xy')
    Z = np.full_like(X, z0, dtype=float)
    return X, Y, Z

def steering_vectors(mics_xyz, X, Y, Z, c, f0):
    """Compute narrowband steering matrix (M x G)."""
    k = 2*np.pi*f0 / c
    gx, gy, gz = X.ravel()[None,:], Y.ravel()[None,:], Z.ravel()[None,:]
    mx, my, mz = mics_xyz[:,0][:,None], mics_xyz[:,1][:,None], mics_xyz[:,2][:,None]
    R = np.sqrt((mx-gx)**2 + (my-gy)**2 + (mz-gz)**2)
    R0 = R[0:1,:]
    phase = np.exp(-1j * k * (R - R0))
    G = phase / np.maximum(R, 1e-6)
    return G, R

def das_power_map(G, p_vec):
    """Conventional DAS power map."""
    M = G.shape[0]
    y = (G.conj().T @ p_vec) / M
    return np.abs(y)**2


In [4]:
# --- Synthetic signal generation ---

def simulate_sinusoids(mics_xyz, sources, f0, fs, T, c=343.0, amp=0.1):
    """Generate time-domain pressures for given sources."""
    t = np.arange(0, T, 1/fs)
    M = mics_xyz.shape[0]
    P = np.zeros((t.size, M), dtype=float)
    for s in sources:
        sx, sy, sz = s['x'], s['y'], s['z']
        a = s.get('amp', amp)
        phi = s.get('phase', 0.0)
        R = np.sqrt((mics_xyz[:,0]-sx)**2 + (mics_xyz[:,1]-sy)**2 + (mics_xyz[:,2]-sz)**2)
        tau = R / c
        A = a / np.maximum(R, 1e-6)
        P += (A[None,:] * np.sin(2*np.pi*f0*(t[:,None] - tau[None,:]) + phi))
    return t, P


In [5]:
# --- Load microphone positions and visualize ---

mics_xyz = load_mic_positions(MIC_XLSX)
M = mics_xyz.shape[0]
print(f"Loaded {M} microphones.")

plt.figure()
plt.scatter(mics_xyz[:,0], mics_xyz[:,1], facecolors='none', edgecolors='k')
plt.gca().set_aspect('equal', 'box')
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.title("Microphone Array Geometry")
plt.show()


ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

In [ ]:
# --- Compute grid and steering vectors ---

X, Y, Z = make_grid(grid_extent, grid_step, focus_z)
G, _ = steering_vectors(mics_xyz, X, Y, Z, c, f0)

print(f"Steering matrix shape: {G.shape} (M={M}, G={X.size})")


In [ ]:
# --- DAS on recorded data ---

try:
    t_rec, P_rec = load_pressures(PRESSURE_XLSX)
    fs_rec = estimate_fs(t_rec)
    print(f"Recorded signals: N={P_rec.shape[0]}, M={P_rec.shape[1]}, fs≈{fs_rec:.1f} Hz")

    # Complex mic vector at f0
    p_vec_rec = complex_projection_at_freq(P_rec, fs_rec, f0)
    Pmap_rec = das_power_map(G, p_vec_rec).reshape(X.shape)

    plt.figure()
    plt.imshow(10*np.log10(np.maximum(Pmap_rec, 1e-20)),
               extent=[X.min(), X.max(), Y.min(), Y.max()],
               origin='lower', aspect='equal')
    plt.title("DAS Map @ 2 kHz (Recorded Data)")
    plt.xlabel("x [m]"); plt.ylabel("y [m]")
    plt.colorbar(label="Level [dB]")
    plt.show()
except Exception as e:
    print(f"[Warning] Skipped recorded-data DAS: {e}")


In [ ]:
# --- Synthetic single off-centre source ---

t_syn, P_syn = simulate_sinusoids(
    mics_xyz,
    [{'x':0.1, 'y':0.1, 'z':focus_z, 'amp':src_amp}],
    f0, fs_synth, T_synth, c=c
)

p_vec_syn = complex_projection_at_freq(P_syn, fs_synth, f0)
Pmap_syn = das_power_map(G, p_vec_syn).reshape(X.shape)

plt.figure()
plt.imshow(10*np.log10(np.maximum(Pmap_syn, 1e-20)),
           extent=[X.min(), X.max(), Y.min(), Y.max()],
           origin='lower', aspect='equal')
plt.title("DAS @ 2 kHz — Synthetic Off-centre (0.1, 0.1, 1 m)")
plt.xlabel("x [m]"); plt.ylabel("y [m]")
plt.colorbar(label="Level [dB]")
plt.show()


In [ ]:
# --- Synthetic two-source resolution test ---

# Theoretical Rayleigh resolution Δ ≈ 0.61 λ z / D
lam = c / f0
D_ap = max(np.linalg.norm(mics_xyz[i,:2]-mics_xyz[j,:2]) for i,j in combinations(range(M),2))
delta_theory = 0.61 * lam * focus_z / D_ap
print(f"Theoretical Rayleigh Δ ≈ {delta_theory*100:.1f} cm (λ={lam:.3f} m, D={D_ap:.3f} m)")

sep = delta_theory * 1.2  # slightly above theoretical
t_two, P_two = simulate_sinusoids(
    mics_xyz,
    [{'x':-sep/2,'y':0,'z':focus_z,'amp':src_amp},
     {'x': sep/2,'y':0,'z':focus_z,'amp':src_amp}],
    f0, fs_synth, T_synth, c=c
)
p_vec_two = complex_projection_at_freq(P_two, fs_synth, f0)
Pmap_two = das_power_map(G, p_vec_two).reshape(X.shape)

plt.figure()
plt.imshow(10*np.log10(np.maximum(Pmap_two, 1e-20)),
           extent=[X.min(), X.max(), Y.min(), Y.max()],
           origin='lower', aspect='equal')
plt.title(f"DAS @ 2 kHz — Two Sources (separation ≈ {sep:.3f} m)")
plt.xlabel("x [m]"); plt.ylabel("y [m]")
plt.colorbar(label="Level [dB]")
plt.show()
